In [2]:
import os

import loguru
import polars as pl
from polars.exceptions import ComputeError
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError, OperationalError, DataError
from dotenv import load_dotenv

load_dotenv()
logger = loguru.logger

In [4]:
__connect_args = {
    # 필수 파라미터
    "host": os.getenv("DB_HOST"),  # 데이터베이스 호스트
    "user": os.getenv("DB_USER"),  # 데이터베이스 사용자
    "password": os.getenv("DB_PASSWORD"),  # 사용자 비밀번호
    "database": os.getenv("DB_NAME"),  # 데이터베이스 이름
    
    # 선택적 파라미터
    "port": int(os.getenv("DB_PORT", 3306)),  # 포트 (기본값: 3306)
    "charset": "utf8mb4",  # 문자 인코딩
    "collation": "utf8mb4_unicode_ci",  # 정렬 규칙
    "use_unicode": True,  # 유니코드 사용
    
    # 성능 관련 파라미터
    "pool_size": 5,  # 커넥션 풀 크기
    "pool_timeout": 30,  # 풀 타임아웃(초)
    "pool_recycle": 1800,  # 커넥션 재사용 시간(초)
    
    # SSL 설정 (필요시)
    # "ssl": {
    #     "ca": "/path/to/ca.pem",
    #     "cert": "/path/to/client-cert.pem",
    #     "key": "/path/to/client-key.pem"
    # }
}

In [ ]:
class MysqlConnector:
    def __init__(self, connection_args: dict):
        self.__connect_args = connection_args
        self.__engine = create_engine(
            "mysql+pymysql://:@",
            connect_args=self.__connect_args
        )
    
    def execute(self, query: str) -> pl.DataFrame | None:
        result = None
        
        try:
            with self.__engine.connect() as connection:
                result = pl.read_database(query=text(query), connection=connection)
        except OperationalError as e:
            logger.error(f"데이터베이스 연결 오류: {e}")
        except DataError as e:
            logger.error(f"데이터 형식 오류: {e}")
        except ComputeError as e:
            logger.error(f"Polars 계산 오류: {e}")
        except SQLAlchemyError as e:
            logger.error(f"SQLAlchemy 오류: {e}")
        except Exception as e:
            logger.error(f"Error while executing query: {e}")
        
        return result

In [6]:
connector = MysqlConnector(__connect_args)

In [7]:
connector.execute("SELECT * FROM MY_TABLE")

2024-11-21 00:17:22.042 | ERROR    | __main__:execute:25 - Error while executing query: Connection.__init__() got an unexpected keyword argument 'pool_size'
